## Introduction
JavaScript did not have built-in support for modules. In order to imitate a module like functionality, a number of solutions were created over time. Each fulfilling some and failing at other criterias of being a module. For example, a very basic approach was to just separate different concerns in different JavaScript files:

In [ ]:
// user.js
var users = ["Tyler", "Sarah", "Dan"]

function getUsers() {
    return users
}

// dom.js
function addUserToDOM(name) {
    const node = document.createElement("li")
    const text = document.createTextNode(name)
    node.appendChild(text)

    document.getElementById("users").appendChild(node)
}

var users = getUsers()
for (var i = 0; i < users.length; i++) {
    addUserToDOM(users[i])
}

Then inside html, we would include both the JS files:
```html
<head>
    <script src="user.js"></script>
    <script src="dom.js"></script>
</head>
<body>
</body>
```

Two major flaws:
- order in which the files load
- all variables in both files are in global scope, which means any external code can overwrite it

The above example is just code separation. Now, instead of using the global scope, we can define our own scope object.

In [ ]:
// app.js
var APP = {}

// user.js
function usersWrapper () {
    var users = ["Tyler", "Sarah", "Dan"]

    function getUsers() {
        return users
    }

    APP.getUsers = getUsers
}

usersWrapper()

// dom.js
function domWrapper() {
    function addUserToDOM(name) {
        const node = document.createElement("li")
        const text = document.createTextNode(name)
        node.appendChild(text)

        document.getElementById("users").appendChild(node)
    }

    var users = APP.getUsers()
    for (var i = 0; i < users.length; i++) {
        addUserToDOM(users[i])
    }
}

domWrapper()

Now in our HTML, we need to make following change:
```html
<head>
    <script src="app.js"></script>
    <script src="user.js"></script>
    <script src="dom.js"></script>
</head>
<body>
</body>
```

Now there is only two additions to the global object. Moreover, variables like users is completely protected inside the wrapper function and cannot be modified from outside. In order to remove the ugly wrapper functions we make use of revealing module pattern:

**Revealing Module Pattern:** uses IIFE which returns an object which becomes module. The benefit is that inside these functions are code that can be private in such ways that they'd only be accessible within that function's scope unless the returned object provides methods that can access them somehow.

In [ ]:
 // app.js
var APP = {}

// user.js
(function () {
    var users = ["Tyler", "Sarah", "Dan"]

    function getUsers() {
        return users
    }

    APP.getUsers = getUsers
})()

// dom.js
(function () {
    function addUserToDOM(name) {
        const node = document.createElement("li")
        const text = document.createTextNode(name)
        node.appendChild(text)

        document.getElementById("users").appendChild(node)
    }

    var users = APP.getUsers()
    for (var i = 0; i < users.length; i++) {
        addUserToDOM(users[i])
    }
})()

The advantage is that now we just have one object in global namespace, `APP`. Even though we have reduced the number of items we add to global namespace, the problem of correct order when using the script tags still persists.  

Progress on this front led to *CommonJS* specification (other systems like *AMD* and *UMD* also exist). But there are two problems with CommonJS:
- synchronous loading
- No browser support

To solve the second problem we make use of *Module Bundlers* such as Webpack. A module bundler examines the codebase, looks at all the imports and exports, then intelligently bundles all of your modules together into a single file that the browser can understand.

```
app.js ---> |         |
user.js ->  | Bundler | -> bundle.js
dom.js ---> |         |

```

## ES Modules
ES6 introduced native JavaScript modules. There is exactly one module per file and one file per module.

In [ ]:
// Mulitple named exports (area.js)
export const PI = 3.14;

export function circleArea(radius){
    return PI * radius * radius;
}

Export can also be specified separately:

In [ ]:
const PI = 3.14;

function circleArea(radius){
    return PI * radius * radius;
}

export { PI, circleArea };

To import:

In [ ]:
// Importing
import { PI, circleArea } from 'area.js';

// Or we can assign different name to imported
import { PI as pi, circleArea } from 'area.js';

`as` can also be used while exporting:

In [ ]:
export { PI as pi, circleArea }; // import {pi, circleArea} from ...

We can also import all exported items:

In [ ]:
import * as area from 'area.js'; // whenever we import all we must provide a name using as

console.log(area.circleArea(12));

If our module exports a single value, then we can make use of **default export**:

In [ ]:
// def_export.js
export default function(){
    // ...
}

// def_import.js
import myFunction from 'def_export.js';

It is not necessary to make use of only anonymous function or class:

In [ ]:
// def_export_2.js
export default function Foo(){
    // ...
}

// def_import_2.js
import foo from 'def_export_2.js'; // note that import name is not the
                                   // same. foo =/= Foo. We can keep it
                                   // same if we want
// import Foo from 'def_export_2.js';

Or

In [ ]:
function Foo(){
    // ...
}

export { Foo as default };

A module can export other entities in addition to default export:

In [ ]:
// export.js
export default function foo(){}

export function bar(){}
export var baz = 5;

// import.js
import foo, { bar, baz } from 'export.js'

// Or
import { foo as default, bar, baz } from 'export.js';

The structure of ES6 modules is static, we can’t conditionally import or export things. Module imports are hoisted (internally moved to the beginning of the current scope). Therefore, it doesn’t matter where we mention them in a module and the following code works without any problems:

In [ ]:
foo();

import { foo } from 'my_module';

In [ ]:
// Below code is also not allowed
import something from 'my_modeul' + someVariable;

Imports are *read-only view* on exported entities:

In [ ]:
// counter.js
export var count = 0;
export function increment(){
    count++;
}

// main.js
import { count, increment } from "./counter.js";

console.log("Count before:" + count); // 0
increment();
console.log("Count after:" + count); // 1

Changing an imported value would lead to error

In [ ]:
// main.js
import { count } from "./counter.js";

console.log("Count before:" + count); // 0
count++; // error!

Module forms its own scope. This means `var` keyword doesn't add to the global scope.

Module is evaluated only the first time it is imported:

In [ ]:
// console.log is executed only once

// log.js
console.log("Hello!");

// first.js
import "./log.js";

// second.js
import "./log.js";

In a module `this` is undefined.

### ES Modules and Browser

In browser, we can import module by specifying `type=module`:

In [ ]:
// logger.js
export function log(message, severity) {
    console.log(`${serverity.toUpperCase()}: ${message}`);
}

// index.html
<script type="module">
    import { log } from "./logger.js";
    log("Launched script", "info");
</script>

Or by specifying `src` (CORS policies apply):

In [ ]:
<script type="module" src="https://localhost:8080/main.js" />

`file://` protocol doesn't work because of CORS policy.

In [ ]:
<script type="module" src="./main.js"></script>
// Or
<script type="module" src="file:///C:/path/to/main.js"></script>

Browsers complain with error:
> Access to script at 'file:///C:/path/to/main.js' from origin 'null' has been blocked by CORS policy.

Two `script` tags having `type=module` have their own scopes:

In [ ]:
<script type="module">
  // The variable is only visible in this module script
  let user = "John";
</script>

<script type="module">
  alert(user); // Error: user is not defined
</script>

Module scripts are by default `defer`red. This means that
- these scripts are downloaded in parallel with other resources
- executed when the HTML document is fully loaded
- and in the order the script tags were defined

In [ ]:
<script type="module">
    console.log(document.getElementById("btn").textContent); // Button
</script>

<!-- Compare to regular script below: -->

<script>
  console.log(document.getElementById("btn").textContent); // button is undefined.<button> has not loaded yet
</script>

<button id="btn">Button</button>

Module scripts can also be marked as `async`

In [ ]:
<script type="module" async src="https://analytics.com/main.js" />

Older browsers like IE don't support `type=module`. So we can provide a fallback using `nomodule` (since unknow `type` is ignored):

In [ ]:
<script type="module">
  alert("Runs in modern browsers");
</script>

<script nomodule>
  alert("Modern browsers know both type=module and nomodule, so skip this")
  alert("Old browsers ignore script with unknown type=module, but execute this.");
</script>